In [16]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.ImageDraw as ImageDraw
import PIL.Image as Image
from skimage.measure import compare_ssim
import imutils
import cv2

%matplotlib inline

json_file = open("/home/ankit/Downloads/atg.json", "r", encoding="utf-8")
sh_json= json.load(json_file)
json_file.close()

dots=[]
image = Image.new("RGB", (640, 480))
draw = ImageDraw.Draw(image)

for person in sh_json:
    y=len(sh_json['valid_vertices'])
for x in range(0,y):
    k=(sh_json['valid_vertices'])
    dots.append((k[x][0],k[x][1]))
print(dots)

[(100, 90), (100, 400), (555, 70), (573, 728)]


In [17]:
import cv2
import numpy as np

# original image
# -1 loads as-is so if it will be 3 or 4 channel as the original
image = cv2.imread('/home/ankit/Desktop/Barbi.jpg',-1)
# mask defaulting to black for 3-channel and transparent for 4-channel
# (of course replace corners with yours)
mask = np.zeros(image.shape, dtype=np.uint8)
#roi_corners = np.array([[(10,10), (300,300), (10,300)]], dtype=np.int32)
roi_corners = np.array([dots], np.int32)
# fill the ROI so it doesn't get wiped out when the mask is applied
channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
ignore_mask_color = (255,)*channel_count
cv2.fillPoly(mask, roi_corners, ignore_mask_color)
# from Masterfool: use cv2.fillConvexPoly if you know it's convex

# apply the mask
masked_image = cv2.bitwise_and(image, mask)
# save the result
cv2.imwrite('image_masked.png', masked_image)
cv2.imshow('image_masked.png',masked_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [18]:
# Python 2/3 compatibility
from __future__ import print_function

import numpy as np
import cv2 as cv
import video


def draw_flow(points, flow, step=7.5):
    h,  w = points.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int) # mesh-grid gives ndarrays all of the same dimensions
    fx, fy = flow[y,x].T
    points1 = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2) # vstack is use for to collect the array and make stack of array in vertical form (row wise)
    points1 = np.int32(points1 + 0.5)
    vis = cv.cvtColor(points, cv.COLOR_GRAY2BGR)
    
    #mask = np.zeros(image.shape, dtype=np.uint8)
    #channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
    #ignore_mask_color = (255,)*channel_count
    #roi_corners = np.array([dots], np.int32)
    #cv2.fillPoly(mask, roi_corners, ignore_mask_color)
    # masked_image = cv2.bitwise_and(image, mask)

    #cv2.imwrite('image_masked.png', masked_image)
    #cv2.imshow('image',masked_image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    cv.polylines(vis, points1, True, (0, 100,500))
    for (x1, y1), (_x2, _y2) in points1:
        cv.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis 

def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    return bgr


def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv.remap(img, flow, None, cv.INTER_LINEAR)
    return res


if __name__ == '__main__':
    import sys  
    
    cam = cv2.VideoCapture('/home/ankit/Downloads/opencv-master /samples/data/vtest.avi')
    cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    codec = cv2.VideoWriter_fourcc(*'XVID') 
    output1 = cv2.VideoWriter('/home/ankit/Downloads/opencv-master /samples/python/just.avi', codec, 60.0,(1920,1080))
    
    ret, prev = cam.read() 
    mask2 = np.zeros(prev.shape, dtype=np.uint8)
    roi_corners2 = np.array([dots], np.int32)
    channel_count2 = prev.shape[2]
    ignore_mask_color2 = (255,)*channel_count2
    cv2.fillPoly(mask2, roi_corners2, ignore_mask_color2)
    masked_image2 = cv2.bitwise_and(prev, mask2)
    # save the result
    #cv2.imwrite('image_masked.png', masked_image)
    prevgray = cv2.cvtColor(masked_image2, cv2.COLOR_BGR2GRAY)

    show_hsv = False
    show_glitch = False
    cur_glitch = prev.copy()
    
    while(cam.isOpened()):
        ret, prev = cam.read()
        mask1 = np.zeros(prev.shape, dtype=np.uint8)
        roi_corners1 = np.array([dots], np.int32)
        channel_count1 = prev.shape[2]
        ignore_mask_color1 = (255,)*channel_count1
        cv2.fillPoly(mask1, roi_corners1, ignore_mask_color1)
        masked_image1 = cv2.bitwise_and(prev, mask1)
        # save the result
        #cv2.imwrite('image_masked.png', masked_image)
        gray = cv.cvtColor(masked_image1, cv.COLOR_BGR2GRAY) 
        flow = cv.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) 
        prevgray = gray
        cv.imshow('flow', draw_flow(gray, flow))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
        
        if show_hsv:
            cv.imshow('flow HSV', draw_hsv(flow))
        if show_glitch:
            cur_glitch = warp_flow(cur_glitch, flow)
            cv.imshow('glitch', cur_glitch)
        ch = cv.waitKey(1)
        if ch == 27:
            break
        if ch == ord('1'):
            show_hsv = not show_hsv
            print('HSV flow visualization is', ['off', 'on'][show_hsv])
        if ch == ord('2'):
            show_glitch = not show_glitch
            if show_glitch:
                cur_glitch = img.copy()
            print('glitch is', ['off', 'on'][show_glitch])

    cam.release()
    output1.release()
    cv.destroyAllWindows()
